In [2]:
#
# ISP Endterm Exercise 1
#

# Jupyter Notebook Notebooks For Detection Of Cars

In [3]:
#
# Background
#
# Task 1
# The first task consists of developing an application (exercise 1.1) for detecting and tracking moving 
# cars from a camera recording.
#
# Task 2
# In order to prevent traffic jams, your client is also very interested in the number of cars that go from 
# the city’s downtown to the city centre in peak hours.
#
# Therefore, Task 2 consists of developing a computer vision application (exercise 1.2) for counting the 
# number of cars that go from the city’s downtown to the city centre for a specific time interval.
#

In [4]:
#
# The following references were very helpful in develoing this application :
#
# 1. Object Detection using OpenCV | Python | Tutorial for beginners 2020 - Youtube Video : 
#  https://www.youtube.com/watch?v=RFqvTmEFtOE
#
# 2. Gaurav Rajesh Sahani - OpenCV Implementation - Github repo : https://github.com/GauravSahani1417/OpenCV-Implementaion
#
# 3. How to Perform Motion Detection Using Python : https://www.kdnuggets.com/2022/08/perform-motion-detection-python.html
#
# 4. Object Tracking from scratch with OpenCV and Python : https://www.youtube.com/watch?v=GgGro5IV-cs
#

In [5]:
#
# This method based on getting an initial image then and comparing each subsequent frame with that image
# this should overcome the problem with stationary cars fading away as even stationary objects will represent a  
# difference from the orginal frame 
#

In [9]:
# object detection in Python using OpenCV
%pip install opencv-python

# install the gdown library to utilize the Google Drive download feature
%pip install gdown

# Import the OpenCV and other libraries as required
import cv2
import math
import numpy as np

# Import gdown to download the video file
import gdown

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [11]:
# define some useful functions

# calc centroid from bounding rectangle of contour
def centroidFromRect(x, y, w, h):
    # calculate the centroid of each box, store in centroids & draw circle
    # add an additional value cdx for horizonal speed intialised to zero 
    cx = int((2*x+w)/2)
    cy = int((2*y+h)/2)
    # this is to store the horizontal speed
    cdx = 0
    centroid = (cx,cy,cdx)
    return centroid

# find index of centroid in tracked objects
def indexFromCentroid(centroid, trackedObjects):
    # just test the first 2 values in the tuple i.e. (x,y)
    listOfIndexes = [key  for (key, value) in trackedObjects.items() if value[:2] == centroid[:2]]
    return listOfIndexes

# from reference : https://medium.com/analytics-vidhya/
# tutorial-how-to-scale-and-rotate-contours-in-opencv-using-python-f48be59c35a2
# scale a contour used to make smaller
def scale_contour(cnt, scale):
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    cnt_norm = cnt - [cx, cy]
    cnt_scaled = cnt_norm * scale
    cnt_scaled = cnt_scaled + [cx, cy]
    cnt_scaled = cnt_scaled.astype(np.int32)

    return cnt_scaled

# define function that gets x,y value from centroid that now contains speed
def centerFromCentroid(centroid):
    cx = centroid[0]
    cy = centroid[1]
    return (cx, cy)


In [12]:
# define some initial variables
# Assigning our initial state in the form of variable initialState as None for initial frames  
initialState = None

# object area threshold
minObjectArea = 7000

# object reactangle scale factor
objScale = 0.2

# threshold for deleting object if disapeared
maxDisappeared = 5

# threshold distance for same object on subsequent frames
maxDistance = 40

# threshold for movement left
minDx = -3

# define the media and loop to find the objects
# Set up the detail for using the test mp4 file for car recognition
#video = cv2.VideoCapture("Traffic_Laramie_1.mp4")
#video = cv2.VideoCapture("Traffic_Laramie_2.mp4")

# define code to download the video file from Google Drive
#url = 'https://drive.google.com/uc?id=1xqVY1FRCLVYS9jSNAaRrw1nXhXwWDc7t'
#output = 'Traffic_Laramie_1.mp4'
url = 'https://drive.google.com/uc?id=1Bo-Ngf2XyxRb5bzBWC9PjA2iY-Zphon7'
output = 'Traffic_Laramie_2.mp4'
gdown.download(url, output, quiet=False)

# define the video capture object
#video = cv2.VideoCapture("Traffic_Laramie_1.mp4")
video = cv2.VideoCapture("Traffic_Laramie_2.mp4")

# check if the video is opened correctly
if not video.isOpened():
    video = cv2.VideoCapture(1)
    raise IOError("Cannot Open Video !!!")

# We use VideoCapture function to create the video capture object
#video = cv2.VideoCapture(1)

# initialise the variables to store the object centroids for previous frame
centroidsPrevFrame = []

# initalise a dict to keep tracked objects, disappeared count & id
# disappeared keeps count of how many frames object has disappered
# this could be used to delete and object after it has disappeared for a given number of iterations
trackedObjects = {}
updatedTrackedObjects = {}
disappeared = {}
trackedId = 0

# car counting variables
vidFrameRate = 25 # frames/second from .json file create using using ffprobe
numCars = 0
minutes = 0
carsPerMinute = 0
framesPerMinute = 0
numFrames = 0

# We start an infinite loop and keep reading frames from the webcam until we press 'q'
while video.isOpened():

    ret, image = video.read()
    if not ret:
        continue
        
    # we are only interested in traffic flow on main road i.e. bottom half of screen
    # is the Region Of Interest roi
    height, width, _ = image.shape

    # limit the roi to the bottom half of the screen 
    roi = image[int(height/2):height,0:width]    
    
    # For the detection classifier to work, we need to convert the frame into greyscale
    # limit this to the roi
    gray_image = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
    # To find the changes creating a GaussianBlur from the gray scale image  
    gray_frame = cv2.GaussianBlur(gray_image, (21, 21), 0)
    
    # For the first iteration checking the condition
    # we will assign grayFrame to initalState if is none  
    if initialState is None:  
        initialState = gray_frame
        continue

    # Calculation of difference between static or initial and gray frame we created  
    differ_frame = cv2.absdiff(initialState, gray_frame)
    
    # the change between static or initial background and current gray frame are highlighted
    # this will give any differences above 20 a value of 255 else set it to zero to emphasise contrast
    thresh_frame = cv2.threshold(differ_frame, 20, 255, cv2.THRESH_BINARY)[1]
    
    # dilation will increase the objects area and accentuate features
    dilate_frame = cv2.dilate(thresh_frame, None, iterations = 10)

    # find all contours based on dilated image using most significant contours
    # RETR_EXTERNAL instead of RETR_CCOMP finds only outer contours
    contoursd,_ = cv2.findContours(dilate_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    # this section will attempt to draw the bounding rectangles of the contours to a black mask
    # then repeat process of finding contours on the mask
    # create an empty black mask and a copy
    mask = np.zeros(dilate_frame.shape[:2],dtype=np.uint8)
    
    # fill the mask with bounding rectangles of the contours great than a size threshold
    for cd in contoursd:
        if cv2.contourArea(cd) > minObjectArea:
            # however make bounding box a little smaller to avoid collisions of boxes
            cs = scale_contour(cd, objScale)
            x,y,w,h  = cv2.boundingRect(cs)            
            cv2.rectangle(mask,(x,y),(x+w,y+h),(255),-1)
            
    # find the contours from the mask image
    contours,_ = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
        
    # let us initialise the variables to store the object centroids for this frame
    centroidsThisFrame = []
    
    # make a list of ids updated/added this frame so that objects not updated can be removed
    updated = []

    for contour in contours:
        # draw the bounding box
        # however make bounding box bigger se extend contour back to original size
        ce = scale_contour(contour, 1/objScale)
        x, y, w, h = cv2.boundingRect(ce)
        cv2.rectangle(roi, (x, y), (x+w, y+h), (0, 255, 0), 3)

        # calculate & store centroid of bounding rectangle
        centroid = centroidFromRect(x, y, w, h)
        centroidsThisFrame.append(centroid)
    
    # compare current centroids to previous centroids or tracked objects
    for centroid in centroidsThisFrame:
        for centroidPrev in centroidsPrevFrame:
            distance = math.hypot(centroidPrev[0]-centroid[0],centroidPrev[1]-centroid[1])
            
            #if distance is small then update centroid
            if distance < maxDistance:
                # this is deemed a valid object so either :
                # 1. update the existing tracked object or
                # 2. create a new one with a new one
                # store centroid for current point with a key of it's id
                # get index/id of previous centroid in tracked objects
                #print(f'distance = {distance}')
                index = indexFromCentroid(centroidPrev, trackedObjects)
                
                if not index:
                    # add this index to updated list & create new object
                    updated.append(trackedId)
                    # if it does not exist create new object in dict
                    trackedObjects[trackedId] = centroid
                    # also create a disappeared record in dict
                    disappeared[trackedId] = 0
                    trackedId += 1
                else:
                    # add this index to updated list & update new object
                    updated.append(index[0])
                    # else just update found index with current centroid
                    # but first calculate the dx from the previous centroid
                    dx = centroid[0]-centroidPrev[0]
                    #unpack centroid to a list, assign and then back to a tuple
                    tlist = list(centroid)
                    tlist[2] = dx
                    centroid = tuple(tlist)
                    trackedObjects[index[0]]= centroid
                    
                #found so no need to loop though other centroids
                #continue
    
    # loop through disappeared dict and increment value if not updated    
    for k,v in disappeared.items():
        if k not in updated:
            disappeared[k] += 1
    
    #loop though tracked objects only keeping ones just updated or disappeared for a limited number of iterations    
    for k,v in trackedObjects.items():
        if (k in updated) or (disappeared[k] < maxDisappeared):
            updatedTrackedObjects[k] = v
        else:
            # delete as it is no longer on screen
            # first check if dx is negative and increment number of cars going downtown
            if v[2] < minDx:
                numCars += 1
            disappeared.pop(k, None)
    
    # copy and clear objects array for next iteration
    trackedObjects = updatedTrackedObjects.copy()
    updatedTrackedObjects.clear()
        
    # now display tracked object circle and data
    for objectId, centroid in trackedObjects.items():
        center = centerFromCentroid(centroid)
        cv2.circle(roi, center, 5, (0,0,255), -1)
        cv2.putText(roi, str(objectId), (centroid[0], centroid[1]-7), 0, 1, (255,255,255), 1)
                    
    # make a copy of current centroids for next interation
    centroidsPrevFrame = centroidsThisFrame.copy()
  
    # show the dilated frame, object should be bigger
    cv2.imshow('Dilated Frame', dilate_frame)
    
    # show the mask
    cv2.imshow('Mask Frame', mask)
    
    # increment number of frames and calculate stats
    numFrames += 1
    minutes = numFrames / (vidFrameRate * 60)
    carsPerMinute = int(numCars / minutes)

    # display running totals
    cv2.rectangle(image, (10, 50), (460, 180), (255, 255, 255), 3)
    cv2.putText(image, f'Total Number Of Cars : {numCars}', (20,100), 0, 1, (255,255,255), 3)
    cv2.putText(image, f'Cars Per Minute       : {carsPerMinute}', (20,140), 0, 1, (255,255,255), 3)

    # show the video display image
    cv2.imshow('Car Detector', image)    
    
    # Press 'q' to 'quit'
    if cv2.waitKey(30) & 0xFF == ord("q"):
        break

video.release()
cv2.destroyAllWindows()

Downloading...
From: https://drive.google.com/uc?id=1Bo-Ngf2XyxRb5bzBWC9PjA2iY-Zphon7
To: c:\Users\DELL\Desktop\CS DEV AREAS COMPLETED\26 ISP Dev Area\ZZZ Endterm Work\ETEx1Clean\Traffic_Laramie_2.mp4
100%|██████████| 66.1M/66.1M [00:15<00:00, 4.16MB/s]
